# Tutorial for loading, training and using customized `neuralg` models
   But first, imports!

In [ ]:
# Append main folder
import sys
sys.path.append("../")

# Supporting packages
import torch
import matplotlib.pyplot as plt
import numpy as np
from dotmap import DotMap
from copy import deepcopy

# Imports from module
from neuralg.io.get_model import get_model
from neuralg.io.save_model import save_model
from neuralg.io.delete_model import delete_model
from neuralg.training.train_model import train_model
from neuralg.utils.set_up_torch import set_up_torch
from neuralg.ops.svd import svd
from neuralg.evaluation.evaluate_model import evaluate_model
from neuralg.evaluation.compute_accuracy import compute_accuracy



## How to load and re-train existing models
This particular tutorial is devoted to the `svd` operation, i.e. finding singular values of matrices. However, the equivalent procedures are applicable to all supported operations in the module. 

The default models in `neuralg` designated to approximate singular values are trained on random square matrices with elements uniformly distributed on [-10,10]. Depending on the user application, the performance on matrices with different properties might not be immediately satisfactory. 

Let's assume that in your application, it is more likely that the matrix elements are realisations of standard gaussian random variables. To this end, the `neuralg` built-in training loop supports fine-tuning the existing models by re-training on such matrices. This tutorial serves as an explanation how to do so. Alternatively, if you already have a generated data set or a more exotic matrix distribution you can simply load the model of choice to fine-tune in your own training loop. 

### Loading a model 
To load a model, you need to specify the target operation and the matrix size. Note that with this call, the matrix size must be available for the target operation. See [README](linktoreadme) for detailed specification on supported sizes. 

In [ ]:
operation = "svd"
matrix_dimension = 20
svd_model = get_model(operation, matrix_dimension) 

Unless you want to define a new model from scratch - in that case, you can instantiate a new model by calling: 

In [ ]:
new_svd_model = get_model(operation,100, new = True)

#### Small pre-training evaluation
As a baseline reference, we look at how the out-of-the-neuralg-box model performs on the target matrices. For this purpose, we consider a predicted collection of singular values $\hat{\sigma}$ to be a correct solution to the problem if it approximates the ground truth $\sigma$ to a given tolerance $\tau$. Specifically, we check that $||\sigma-\hat{\sigma}||_{1} \leq \tau||\sigma||_{1}$ is satisfied. The ground truths are computed with `torch.svdvals()`.

In [ ]:
tol = 0.1 # We set tolerance to 10% 
eval_set_size = 10000 # And evaluate on this many matrices

# The way matrices are generated in the module, it requires a dict or a DotMap passing the batch parameters, e.g. :
test_matrix_parameters = DotMap({"N":eval_set_size, 
                            "operation": operation, 
                            "d": matrix_dimension, 
                            "normal": True, # Default is uniform entries
                            "sigma": 1,     # Standard deviation of normal elements. Default is 10/sqrt(3) 
                            })
svd_model.eval()
errors = evaluate_model(svd_model, test_matrix_parameters) # Compute the L1 errors on the evaluation set
accuracy_before_training = compute_accuracy(tol,errors) # And assess the accuracy

print(f"With a tolerance of {tol:.0%}, evaluated model achieved accuracy {accuracy_before_training:.4}")

### Training a model with the in-place training loop
The model does not generalize particularly well to the target matrices. We will try to improve the accuracy by re-training the model on the target matrices by accessing the in-place training loop.
#### Specifying data generation and training parameters
The way the training loop is implemented, it requires passing training specifications as dicts or dotmaps alongside the model. Specifically, we need to define the properties of the matrix batches to train on and run parameters such as how many iterations the optimisation algorithm should run for and initial learning rate. 

Default optimizer is the Adam algorithm and an exponential learning rate schedule. 

In [ ]:
batch_size = 64
training_matrix_parameters = deepcopy(test_matrix_parameters) # We can just copy the target matrix settings,
training_matrix_parameters.N = batch_size      # But change the number of matrices
training_run_parameters = DotMap({ "epoch": 1, # Number of epochs
                           "iterations": 50, # Batches per epoch
                           "lr": 3e-5} )       # Learning rate

#### Run the training 
The function `train_model()` will return a dotmap with the trained model, alongside the loss logs from the run. 

In [ ]:
set_up_torch("float32", torch_enable_cuda= True) # Enable training on the GPU, if available

svd_model.train()
training_results = train_model(svd_model,training_matrix_parameters, training_run_parameters) # Run training!

#### Post-training evaluation
Now we hope to see some performace improvement for the target matrices.

In [ ]:
trained_svd_model = training_results.model # Get the trained model 
trained_svd_model.eval()
# Same evaluation procedure as pre-training
errors = evaluate_model(trained_svd_model, test_matrix_parameters) # Compute the L1 errors on the evaluation set
accuracy_after_training = compute_accuracy(tol,errors) # Assess accuracy
print(f"With a tolerance of {tol:.0%}, model achieved accuracy {accuracy_after_training:.3} on the evaluation set after training")


### Saving and using a custom model 

#### Save custom model
If the training outcome was satisfactory, you can save the model by naming it and call `save_model()`. This will save the best model state dict from training to a directory in the user project folder called custom_models. If the directory does not exist it will be created. 

In [ ]:
model_name = "svd_standard_normal" # Preferably an informative name
save_model(trained_svd_model, model_name) # Save model state dict 

#### Use it in the module! 
To be able to use the customized model for future singular value approximations, pass the new model name as an optional argument to `svd()`. 

In [ ]:
# Either by defining a lambda function,
svd_standard_normal = lambda x: svd(x,custom_model_name=model_name)

m = torch.randn(matrix_dimension,matrix_dimension) # Sample test matrix

singular_value_predictions = svd_standard_normal(m) # Predict using the defined custom operation

# Or equivalently calling svd() directly with the additional argument 
singular_value_predictions = svd(m,custom_model_name=model_name)



#### Visualize the results

In [ ]:
fig = plt.figure()

### Plot singular value sample
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(np.arange(1,matrix_dimension+1), singular_value_predictions.detach().numpy(),s = 200, color='pink', marker='o', label = "neuralg")
ax.scatter(np.arange(1,matrix_dimension+1), torch.linalg.svdvals(m).detach().numpy(), s = 200, color='purple', marker='x', label = "torch")
ax.set_title("Sample singular value approximation", fontsize = 18)
ax.legend(fontsize = 16, loc = "best")
ax.set_xticks(np.arange(0, matrix_dimension, 1) + 1)
ax.set_xlabel(" Singular value ", fontsize = 16);

### Deleting models
If a model is now longer of use or the module is getting large, you might want to delete a model by calling`delete_model()`. This will clear the folder custom_models from the requested model state dict.

In [ ]:
delete_model(model_name) # Delete the customized saved model

### Even more customizing: Adding your own model ideas
#### Defining a custom model class


In [ ]:
class DiagConvNet(torch.nn.Module):
    def __init__(self, hidden_layers, filters, kernel_size):
        super(DiagConvNet, self).__init__()
        self.net = []
        self.net.append(torch.nn.Conv2d(1, filters, kernel_size, padding="same"))
        self.net.append(torch.nn.BatchNorm2d(filters))
        self.net.append(torch.nn.ReLU())
        for i in range(hidden_layers - 1):
            self.net.append(
                torch.nn.Conv2d(filters, filters, kernel_size, padding="same")
            )
            self.net.append(torch.nn.BatchNorm2d(filters))
            self.net.append(torch.nn.ReLU())

        self.net.append(torch.nn.Conv2d(filters, 1, kernel_size, padding="same"))
        self.net = torch.nn.Sequential(*self.net)

    def forward(self, x):
        out = self.net(x)
        out = torch.diagonal(out, 0, 2, 3)
        return out

model = DiagConvNet(5,32,3)
save_model(model,"prototype_model", custom_class=True)

#### Using a customized model type in the module

In [ ]:
matrix = torch.randn(1,1,matrix_dimension,matrix_dimension) # Dealing with a convolutional network, we add a batch and channel dimension
singular_value_predictions = svd(matrix, custom_model_name="prototype_model", custom_model_class = True)